In [210]:
import pandas as pd
import subprocess
import numpy as np
from math import sin, cos, sqrt, atan2, radians # Para calcular distacias entre 2 ubicaciones (lat, long)
from datetime import datetime #Para categorizar las horas
import pandas_profiling


In [194]:
#Rutas a los csv generados con OpenRefine
_ruta_motivos_viaje='Viajes_bogota_preprocessing-sinpicos.csv' 
_ruta_troncales_transmi = 'troncales.csv'

#Ruta del archivo csv de salida
_ruta_salida='inprocedings-preprocessed.csv' 

# Leer el archivo

In [195]:

_df_viajes= pd.read_csv(_ruta_motivos_viaje)
_df_troncales= pd.read_csv(_ruta_troncales_transmi)


_df_viajes.columns = ['motivo viaje','hora inicio', 'hora fin', 'latitud origen', 'latitud destino', 'longitud origen', 
                      'longitud destino', 'duracion viaje', 'dia habil']
_df_troncales.columns = ['codigo troncal', 'longitud', 'latitud']


#Reemplazar las comillas dobles por simples en los nombres. 
#_df['nombre']=_df['nombre'].str.replace('"','-')


In [196]:
_df_viajes

,motivo viaje,hora inicio,hora fin,latitud origen,latitud destino,longitud origen,longitud destino,duracion viaje,dia habil
0,Asuntos de Trabajo,17:48:00,18:21:00,4.61634,4.63389,-74.17084,-74.07574,00:33:00,1
1,Volver a casa,22:30:00,23:17:00,4.63389,4.61634,-74.07574,-74.17084,00:47:00,1
2,Trabajar,07:00:00,09:00:00,4.58677,4.74682,-74.07927,-74.09617,02:00:00,0
3,Volver a casa,17:30:00,18:40:00,4.74682,4.64113,-74.09617,-74.16209,01:10:00,0
4,Trabajar,08:20:00,09:30:00,4.62274,4.75203,-74.14933,-74.04790,01:10:00,0
5,Volver a casa,17:42:30,18:52:40,4.75203,4.62274,-74.04790,-74.14933,01:10:10,0
6,Estudiar,07:10:00,08:30:00,4.64404,4.64892,-74.16586,-74.08150,01:20:00,0
7,Volver a casa,14:10:00,15:30:00,4.64892,4.64404,-74.08150,-74.16586,01:20:00,0
8,Estudiar,10:19:00,11:05:00,4.64404,4.64379,-74.16586,-74.06206,00:46:00,0
9,Volver a casa,14:36:00,15:36:00,4.64379,4.64404,-74.06206,-74.16586,01:00:00,0


In [197]:
_df_troncales

,codigo troncal,longitud,latitud
0,F,-74.125316,4.622236
1,K,-74.094985,4.643662
2,C,-74.079477,4.710572
3,D,-74.085459,4.686494
4,B,-74.052794,4.713528
5,E,-74.078407,4.642567
6,J,-74.073915,4.602135
7,G,-74.132869,4.599890
8,H,-74.103172,4.568386
9,H,-74.131901,4.568971


# Calcular distancias
Método que calcula las distancias dadas la latitud y la longitud de 2 ubicaciones

In [198]:
def distancia(latitud1, longitud1, latitud2, longitud2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(latitud1)
    lon1 = radians(longitud1)
    lat2 = radians(latitud2)
    lon2 = radians(longitud2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c


# Combinar datasets
Se consolida en un solo CSV las 2 bases de datos a usar

In [199]:
#Motivos de viaje
_latitudes_origen =_df_viajes['latitud origen']
_latitudes_destino =_df_viajes['latitud destino']
_longitudes_origen =_df_viajes['longitud origen']
_longitudes_destino =_df_viajes['longitud destino']

#Troncales transmi
_latitud_troncales = _df_troncales['latitud']
_longitud_troncales = _df_troncales['longitud']
_troncal = _df_troncales['codigo troncal']

In [200]:
def insertarNuevoValor(lat, lon, name):
    troncales = []
    for i in range(len(lat)):
        distancia_minima = -1
        troncal = ""
        
        for y in range(len(_latitud_troncales)):
            dist = distancia(lat[i], lon[i], _latitud_troncales[y], _longitud_troncales[y])
            if distancia_minima == -1 or distancia_minima > dist:
                distancia_minima = dist
                troncal = _troncal[y]
                
        troncales.append(troncal)
    _df_viajes[name] = troncales

In [201]:
insertarNuevoValor(_latitudes_origen, _longitudes_origen, 'troncal origen')
insertarNuevoValor(_latitudes_destino, _longitudes_destino, 'troncal destino')

In [203]:
_df_viajes

,motivo viaje,hora inicio,hora fin,latitud origen,latitud destino,longitud origen,longitud destino,duracion viaje,dia habil,troncal origen,troncal destino
0,Asuntos de Trabajo,17:48:00,18:21:00,4.61634,4.63389,-74.17084,-74.07574,00:33:00,1,G,A
1,Volver a casa,22:30:00,23:17:00,4.63389,4.61634,-74.07574,-74.17084,00:47:00,1,A,G
2,Trabajar,07:00:00,09:00:00,4.58677,4.74682,-74.07927,-74.09617,02:00:00,0,L,C
3,Volver a casa,17:30:00,18:40:00,4.74682,4.64113,-74.09617,-74.16209,01:10:00,0,C,F
4,Trabajar,08:20:00,09:30:00,4.62274,4.75203,-74.14933,-74.04790,01:10:00,0,F,B
5,Volver a casa,17:42:30,18:52:40,4.75203,4.62274,-74.04790,-74.14933,01:10:10,0,B,F
6,Estudiar,07:10:00,08:30:00,4.64404,4.64892,-74.16586,-74.08150,01:20:00,0,F,E
7,Volver a casa,14:10:00,15:30:00,4.64892,4.64404,-74.08150,-74.16586,01:20:00,0,E,F
8,Estudiar,10:19:00,11:05:00,4.64404,4.64379,-74.16586,-74.06206,00:46:00,0,F,A
9,Volver a casa,14:36:00,15:36:00,4.64379,4.64404,-74.06206,-74.16586,01:00:00,0,A,F


# Categorizar según horas

### De lunes a sábado las categorías son las siguientes:

valle 1: 4:00 - 5:59 AM

pico 1: 6:00 - 8:29 

valle 2: 8:30 - 9:29 

pico 2: 9:30 - 15:29

valle 3: 15:30 - 16:29

pico 3: 16:30 - 19:29

valle 4: 19:30 - fin de operación 


### Para domingos y festivos:

valle dyf: Todo el día


In [204]:
horas_limite = {
    'valle_1': '04:00:00',
    'pico_1': '06:00:00',
    'valle_2': '08:30:00',
    'pico_2': '09:30:00',
    'valle_3': '15:30:00',
    'pico_3': '16:30:00',
    'valle_4': '19:30:00',
    'fin_dia': '00:00:00'
}

In [205]:
formato_hora_limite = '%H:%M:%S'    

def categoria_por_hora(hora):
    valle_1 = datetime.strptime( horas_limite['valle_1'], formato_hora_limite).time()
    pico_1 = datetime.strptime( horas_limite['pico_1'], formato_hora_limite).time()
    valle_2 = datetime.strptime( horas_limite['valle_2'], formato_hora_limite).time()
    pico_2 = datetime.strptime( horas_limite['pico_2'], formato_hora_limite).time()
    valle_3 = datetime.strptime( horas_limite['valle_3'], formato_hora_limite).time()
    pico_3 = datetime.strptime( horas_limite['pico_3'], formato_hora_limite).time()
    valle_4 = datetime.strptime( horas_limite['valle_4'], formato_hora_limite).time()
    fin_dia = datetime.strptime( horas_limite['fin_dia'], formato_hora_limite).time()
    
    categoria = 'no-definida'
    if hora >= valle_1 and hora < pico_1:
        categoria = 'Valle-1'
    elif hora >= pico_1 and hora < valle_2:
        categoria = 'Pico-1'
    elif hora >= valle_2 and hora < pico_2:
        categoria = 'Valle-2'
    elif hora >= pico_2 and hora < valle_3:
        categoria = 'Pico-2'
    elif hora >= valle_3 and hora < pico_3:
        categoria = 'Valle-3'
    elif hora >= pico_3 and hora < valle_4:
        categoria = 'Pico-3'
    elif hora >= valle_4 and hora < fin_dia:
        categoria = 'Valle-4'
    
    return categoria

In [206]:
_horas_iniciales =_df_viajes['hora inicio']
categorias_horas_iniciales = []

for hora in _horas_iniciales:
    hora_final = datetime.strptime(hora, formato_hora_limite).time()
    categorias_horas_iniciales.append(categoria_por_hora(hora_final))

_df_viajes['categoria hora inicio'] = categorias_horas_iniciales

In [207]:
#Borro columnas que ya no necesito
_df_viajes = _df_viajes.drop('hora inicio', 1)
_df_viajes = _df_viajes.drop('latitud origen', 1)
_df_viajes = _df_viajes.drop('latitud destino', 1)
_df_viajes = _df_viajes.drop('longitud origen', 1)
_df_viajes = _df_viajes.drop('longitud destino', 1)

In [208]:
_df_viajes

,motivo viaje,hora fin,duracion viaje,dia habil,troncal origen,troncal destino,categoria hora inicio
0,Asuntos de Trabajo,18:21:00,00:33:00,1,G,A,Pico-3
1,Volver a casa,23:17:00,00:47:00,1,A,G,no-definida
2,Trabajar,09:00:00,02:00:00,0,L,C,Pico-1
3,Volver a casa,18:40:00,01:10:00,0,C,F,Pico-3
4,Trabajar,09:30:00,01:10:00,0,F,B,Pico-1
5,Volver a casa,18:52:40,01:10:10,0,B,F,Pico-3
6,Estudiar,08:30:00,01:20:00,0,F,E,Pico-1
7,Volver a casa,15:30:00,01:20:00,0,E,F,Pico-2
8,Estudiar,11:05:00,00:46:00,0,F,A,Pico-2
9,Volver a casa,15:36:00,01:00:00,0,A,F,Pico-2


# Categorizar duración de viaje

Corto: 00:30:00 o menos

Medio: 6:00 - 8:29 

Largo: 8:30 - 9:29 

pico 2: 9:30 - 15:29

valle 3: 15:30 - 16:29

pico 3: 16:30 - 19:29

valle 4: 19:30 - fin de operación 


### Para domingos y festivos:

valle dyf: Todo el día



In [211]:
pandas_profiling.ProfileReport(_df_viajes)

NameError: name '_converter' is not defined

# Agrupar los autores por paper

In [7]:
_grouped=_df.groupby('paper')['nombre'].apply(lambda x: '|'.join(x)).reset_index() 


In [8]:
_grouped

,paper,nombre
0,http://dblp.l3s.de/d2r/resource/publications/c...,Mary_Jane_Irwin|Tao_Zhang|Yuan_Xie_0001
1,http://dblp.l3s.de/d2r/resource/publications/c...,Tao_Zhang|Yan_Chen|Yuan_Xie_0001
2,http://dblp.l3s.de/d2r/resource/publications/c...,Hanqing_Lu|Tieniu_Tan|Yan_Zhang
3,http://dblp.l3s.de/d2r/resource/publications/c...,Fatos_Xhafa|Leonard_Barolli|Makoto_Takizawa
4,http://dblp.l3s.de/d2r/resource/publications/c...,Fatos_Xhafa|Leonard_Barolli|Makoto_Takizawa
5,http://dblp.l3s.de/d2r/resource/publications/c...,Hui_Wang|Tao_Wang|Zhiyong_Feng
6,http://dblp.l3s.de/d2r/resource/publications/c...,Bo_Zhang|Xiaokang_Yang|Yi_Xu
7,http://dblp.l3s.de/d2r/resource/publications/c...,Aboul_Ella_Hassanien|Ajith_Abraham|Václav_Snásel
8,http://dblp.l3s.de/d2r/resource/publications/c...,Li_Zhang|Wei_Li|Ying_Zhang
9,http://dblp.l3s.de/d2r/resource/publications/c...,Gang_Liu|Jian_Wang|Jun_Guo


# Obtener una columna por cada autor

In [9]:
#Transforma el dataset para obtener una columna binaria por cada autor. 
#Esto puede tardar bastante tiempo, dependiendo del tamaño del dataset.
_dummies=_grouped['nombre'].str.get_dummies('|')

In [11]:
#Agregar al dataset las columnas agrupadas
_df=pd.concat([_grouped, _dummies],axis=1)

# Exportar a csv

In [12]:

_df.to_csv(_ruta_salida, index=False, encoding='utf8' )

# Generar el arff a partir del csv

In [13]:

with open(_arff_path, "w") as arff:
    p=subprocess.Popen(['java','-classpath',_weka,"weka.core.converters.CSVLoader", _ruta_salida,"-E", '"',"-S", "1-2",'-N','3-last', '-B','56000', '-L','3-last:0,1'],stdout=arff)
    p.wait()
